In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import DBSCAN

C:\Users\Lenovo\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
clusters_init = open('wiki-topcats-categories.txt','rt')
filtered_data = pd.read_csv('filtered-wiki-topcats_f.txt', sep=" ", header=None) 
clusters_formed = pd.read_csv('clusters_f.csv', header = None)
unique_nodes = set(filtered_data[0].values).union(set(filtered_data[1].values))
number_clusters = []
cluster_category = []
nodes_indexes = []
categories = []
with open('wiki-topcats-categories.txt','rt') as f: 
    node2cat = {}
    for line in f:
        name, lines = line.split(';')
        nodes = list(map(int,(lines.split())))
        count_edges = len(nodes)
        nodes = list(filter(lambda x: int(x) in unique_nodes, nodes))
        category_name = name.split(':')[1]
        if len(nodes) != 0:
            cluster_category.append(category_name)
            number_clusters.append(len(nodes))
            nodes_indexes.append(nodes)
        for node in nodes:
            if node in node2cat:
                node2cat[node].append(category_name)
            else:
                node2cat[node] = [category_name]
        categories.append([category_name, count_edges])

### Statistics

In [3]:
df_init = pd.DataFrame(list(zip(cluster_category, number_clusters)), columns = ['topic', 'number of nodes per cluster'])
stat_init = df_init.describe()
stat_algo = clusters_formed.groupby(clusters_formed[1]).count().describe()
print(stat_init)
print(stat_algo)

       number of nodes per cluster
count                 15221.000000
mean                     31.724131
std                     224.723494
min                       1.000000
25%                       5.000000
50%                      15.000000
75%                      34.000000
max                   26562.000000
                  0
count     71.000000
mean    2112.690141
std    11106.724215
min        1.000000
25%        2.000000
50%        9.000000
75%      116.000000
max    85164.000000


In [4]:
#Statistics for outliers

count_in_cluster = clusters_formed.groupby(clusters_formed[1]).count()
outliers = ((count_in_cluster==1) | (count_in_cluster > 50000))
stat_outliers = count_in_cluster[~outliers].describe()
print(stat_outliers)

                  0
count     57.000000
mean    1137.263158
std     5387.534178
min        2.000000
25%        6.000000
50%       13.000000
75%      154.000000
max    30752.000000


### Page names of the clusters

In [5]:
page_name = {int(line.split()[0]):" ".join(line.split()[1:]) for line in open("wiki-topcats-page-names.txt")}
max_cluster_names = clusters_formed.groupby(clusters_formed[1]).groups[279122].map(page_name)
print(max_cluster_names)

Index(['Chiasmal syndrome', 'Pinakion',
       'LyndonHochschildSerre spectral sequence', 'Zariski's main theorem',
       'FultonHansen connectedness theorem', 'Cayley's ruled cubic surface',
       'Annulus theorem', 'Bing's recognition theorem', 'BergmanWeil formula',
       'Gladden, Missouri',
       ...
       'The Temple (Old Orchard Beach, Maine)', 'Octagonal Poultry House',
       'Follen Church Society-Unitarian Universalist', 'Falcon Tabernacle',
       'Dunwoody Village', 'Banks & Shane', 'In From the Night',
       'Keiji Fukuda', 'Firehouse No. 4 (Plainfield, New Jersey)',
       'Engine Company No. 3'],
      dtype='object', length=85164)


In [61]:
#Making id_node and id_category for some further usage

ids_categories = {idx:cat for idx,cat in enumerate(cluster_category)}
idx_nodes = []
for cat in ids_categories.keys():
    for node in nodes_indexes[cat]:
        idx_nodes.append((node,cat))
df_filtered = pd.DataFrame(idx_nodes, columns = ['id_node', 'id_category'])

#### Clusters explanation

In [8]:
res = clusters_formed.groupby(clusters_formed[1])
cluster_id = 279122
descriptive_categories = {}
for i in res.groups[cluster_id]:
    node_id = clusters_formed.iloc[i][0]
    categories_for_node = node2cat[node_id]
    for category in categories_for_node:
        if category in descriptive_categories:
            descriptive_categories[category] += 1
        else:
            descriptive_categories[category] = 1
sum_to_norm = len(res.groups[cluster_id])
for i in descriptive_categories:
    descriptive_categories[i] = descriptive_categories[i]/sum_to_norm
descriptive_categories = pd.DataFrame([(k, v) for k,v in descriptive_categories.items()])
descriptive_categories.sort_values(by=[1],ascending=False)

,0,1
14,Living_people,0.180405
87,Harvard_University_alumni,0.014443
69,Year_of_birth_missing_(living_people),0.014126
96,Article_Feedback_Pilot,0.012106
38,American_military_personnel_of_World_War_II,0.008102
344,American_Jews,0.006857
12,American_novelists,0.006799
525,People_from_New_York_City,0.006317
248,National_Association_of_Independent_Colleges_a...,0.006305
91,Columbia_University_alumni,0.006071


### Anomaly Detection

In [9]:
clusters_counts = np.unique(clusters_formed.values[:, 1],
                     return_counts=True)[1]
clusters = np.unique(clusters_formed.values[:, 1],
                     return_counts=True)[0]
mean = np.mean(clusters_counts, axis=0)
std = np.std(clusters_counts, axis=0)
anomalies = np.any((clusters_counts < mean - 2*std,
                              clusters_counts > mean + 2*std), axis=0)
print("Detecting anomalies with 2-sigma")
print("Anomalies clusters sizes", clusters_counts[anomalies])
print("Anomalies clusters IDs", clusters[anomalies])
print()
model = DBSCAN(min_samples=4, eps=500, p=1)
model.fit(clusters_counts.reshape((-1, 1)))
anomalies = model.labels_ == -1
print("Detecting anomalies with DBSCAN")
print("Anomalies clusters sizes", clusters_counts[anomalies])
print("Anomalies clusters IDs", clusters[anomalies])

Detecting anomalies with 2-sigma
Anomalies clusters sizes [85164 27360 30752]
Anomalies clusters IDs [ 279122 1181827 1205356]

Detecting anomalies with DBSCAN
Anomalies clusters sizes [ 1392 85164 27360 30752]
Anomalies clusters IDs [   6283  279122 1181827 1205356]
